In [1]:
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm
import ta

TRAIN_CSV = r'C:\Users\e0817820\Desktop\tokka\data\raw\train.csv'
TRAIN_COMBINE_CSV = r'C:\Users\e0817820\Desktop\tokka\data\raw\train_combine.csv'
TEST_CSV = r'C:\Users\e0817820\Desktop\tokka\data\raw\test.csv'
ADDITIONAL_TRAIN_CSV = r'C:\Users\e0817820\Desktop\tokka\data\raw\add_train.csv'
MODELS_DIR = r'C:\Users\e0817820\Desktop\tokka\models'

C:\Users\e0817820\AppData\Local\Temp\ipykernel_32020\1140990513.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
use_additional_train = True

In [3]:
train = pd.read_csv(TRAIN_CSV)
lower_limit = train['timestamp'].min()
upper_limit = train['timestamp'].max()
if use_additional_train: 
    additional_train = pd.read_csv(ADDITIONAL_TRAIN_CSV)
    train = pd.concat([train, additional_train], axis=0)
    train.to_csv(TRAIN_COMBINE_CSV, index=False)
    upper_limit = train['timestamp'].max()
    del additional_train
if 'id' in train.columns:
    train.drop(columns=['id'], inplace=True)
if 'Unnamed: 0' in train.columns:
    train.drop(columns=['Unnamed: 0'], inplace=True)
train.dropna(inplace=True)

In [4]:
print(train.shape)

(13790220, 12)


In [5]:
#check nan
print(train.isna().sum())

timestamp             0
open                  0
high                  0
low                   0
close                 0
volume                0
quote_asset_volume    0
number_of_trades      0
taker_buy_volume      0
taker_sell_volume     0
symbol                0
log_return            0
dtype: int64


In [6]:
train.tail(10)

,timestamp,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_volume,taker_sell_volume,symbol,log_return
2747300,1707868140000,2637.2000,2638.47000,2637.20000,2638.08000,126.21450,332969.268197,411,94.46690,249209.847775,ETH,0.000724
2747301,1707868140000,0.1269,0.12690,0.12688,0.12689,122083.50000,15491.357183,68,21201.80000,2690.504121,TRX,-0.000158
2747302,1707868140000,39.7000,39.73000,39.70000,39.71000,815.18000,32368.852000,36,802.40000,31861.265500,AVAX,0.000755
2747303,1707868140000,0.0810,0.08102,0.08100,0.08101,68848.00000,5576.742480,25,65382.00000,5295.961820,DOGE,0.000494
2747304,1707868140000,324.6000,324.80000,324.60000,324.70000,346.54800,112524.034800,63,340.71700,110631.208200,BNB,0.000000
2747305,1707868140000,19.8450,19.84500,19.83200,19.83500,1436.34000,28496.506600,63,613.28000,12167.366150,LINK,0.001763
2747306,1707868140000,49639.9900,49657.76000,49639.99000,49650.79000,13.48601,669588.905333,597,8.41712,417914.220207,BTC,0.000982
2747307,1707868140000,112.3700,112.47000,112.36000,112.41000,1089.41000,122477.842300,161,405.66000,45599.503900,SOL,0.000623
2747308,1707868140000,0.5240,0.52420,0.52390,0.52400,271998.00000,142535.649800,109,131863.00000,69098.642300,XRP,0.001335
2747309,1707868140000,0.5444,0.54480,0.54440,0.54470,6657.20000,3625.256210,40,3959.20000,2155.932210,ADA,0.000367


In [7]:
list_cryptocurrencies = ['AVAX', 'ADA', 'SOL', 'BNB', 'TRX', 'DOGE', 'LINK', 'XRP', 'BTC', 'ETH']

In [8]:
combined_train = pd.DataFrame()
combined_train[train.columns] = 0
for id in tqdm(range(10)):
    combined_train = combined_train.merge(train.loc[train["symbol"] == list_cryptocurrencies[id], ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'log_return']].copy(), on="timestamp", how='outer', suffixes=['', "_"+list_cryptocurrencies[id]])

combined_train = combined_train.drop(train.columns.drop("timestamp"), axis=1)
combined_train = combined_train.sort_values('timestamp', ascending=True)
combined_train = combined_train.fillna(method='ffill')
display(combined_train.head())

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:17<00:00,  1.79s/it]
C:\Users\e0817820\AppData\Local\Temp\ipykernel_32020\3759169524.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_train = combined_train.fillna(method='ffill')


,timestamp,open_AVAX,high_AVAX,low_AVAX,close_AVAX,volume_AVAX,log_return_AVAX,open_ADA,high_ADA,low_ADA,...,low_BTC,close_BTC,volume_BTC,log_return_BTC,open_ETH,high_ETH,low_ETH,close_ETH,volume_ETH,log_return_ETH
0,1625097600000,11.940,11.962,11.940,11.950,714.795,-0.008066,1.3853,1.3862,1.3821,...,34966.09,34980.47,100.833562,-0.003030,2275.68,2275.93,2269.77,2270.06,699.78407,-0.007508
1,1625097660000,11.946,11.949,11.929,11.929,420.382,-0.008503,1.3822,1.3830,1.3787,...,34974.08,34974.51,53.304605,-0.003791,2270.03,2274.49,2267.06,2267.25,709.32090,-0.008393
2,1625097720000,11.909,11.914,11.894,11.900,127.602,-0.007761,1.3788,1.3791,1.3766,...,34901.00,34951.50,102.614870,-0.004600,2267.03,2268.89,2262.71,2266.11,913.31677,-0.009965
3,1625097780000,11.908,11.908,11.896,11.900,107.587,-0.006830,1.3771,1.3785,1.3765,...,34946.43,34950.00,30.444503,-0.003673,2266.11,2268.44,2263.00,2263.01,383.03679,-0.007554
4,1625097840000,11.909,11.919,11.909,11.917,114.725,-0.009782,1.3768,1.3804,1.3768,...,34949.99,34962.89,17.500656,-0.003813,2263.01,2268.86,2263.01,2265.45,527.44357,-0.008712


In [9]:
print(combined_train.shape)

(1379022, 61)


In [10]:
def bollinger_mavg(close, window=20):
    """Calculate the middle Bollinger Band."""
    return close.rolling(window=window).mean()

def bollinger_hband(close, window=20, k=2):
    """Calculate the upper Bollinger Band."""
    mavg = bollinger_mavg(close, window)
    mstd = close.rolling(window=window).std()
    return mavg + (mstd * k)

def bollinger_lband(close, window=20, k=2):
    """Calculate the lower Bollinger Band."""
    mavg = bollinger_mavg(close, window)
    mstd = close.rolling(window=window).std()
    return mavg - (mstd * k)

def bollinger_wband(close, window=20, k=2):
    """Calculate the width of the Bollinger Bands."""
    hband = bollinger_hband(close, window, k)
    lband = bollinger_lband(close, window, k)
    mavg = bollinger_mavg(close, window)
    return ((hband - lband) / mavg) * 100

def exponential_moving_average(close, span=20):
    """Calculate the Exponential Moving Average (EMA)."""
    return close.ewm(span=span, adjust=False).mean()

def relative_strength_index(close, window=14):
    """Calculate the Relative Strength Index (RSI)."""
    delta = close.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi
  
def stochastic_oscillator(high, low, close, window=14):
    """Calculate the Stochastic Oscillator."""
    low_min = low.rolling(window=window).min()
    high_max = high.rolling(window=window).max()
    k = ((close - low_min) / (high_max - low_min)) * 100
    return k

def williams_r(high, low, close, window=14):
    """Calculate Williams %R."""
    high_max = high.rolling(window=window).max()
    low_min = low.rolling(window=window).min()
    r = -100 * ((high_max - close) / (high_max - low_min))
    return r
  
def rate_of_change(close, window=14):
    """Calculate the Rate of Change (ROC)."""
    roc = ((close - close.shift(window)) / close.shift(window)) * 100
    return roc

In [11]:
def delay(x, d):
    """Value of x d days ago"""
    return x.shift(d)

def correlation(x, y, d):
    """Time-series correlation of x and y for the past d days"""
    return x.rolling(window=d).corr(y)

def delta(x, d):
    """Today's value of x minus the value of x d days ago"""
    return x.diff(d)

def ts_max(x, d):
    """Time-series max over the past d days"""
    return x.rolling(window=d).max()

def ts_min(x, d):
    """Time-series min over the past d days"""
    return x.rolling(window=d).min()

In [12]:
def getFeatures(df):
    lags_normal = [15, 40, 150, 300]
    lags_ema = [15, 50, 160, 300]
    lags_rsi = [15, 50, 160, 300]
    lags_bbands = [15, 50, 160, 300]
    for id in range(10):   
        symbol = list_cryptocurrencies[id] 
        for lag in lags_normal:
            df[f'log_close/mean_{lag}_{symbol}'] = np.log( np.array(df[f'close_{symbol}']) /  np.roll(np.append(np.convolve( np.array(df[f'close_{symbol}']), np.ones(lag)/lag, mode="valid"), np.ones(lag-1)), lag-1)  )
            df[f'log_return_{lag}_{symbol}'] = np.log( np.array(df[f'close_{symbol}']) /  np.roll(np.array(df[f'close_{symbol}']), lag)  )
    for lag in lags_normal:
        df[f'mean_close/mean_{lag}'] =  np.mean(df.iloc[:,df.columns.str.startswith(f'log_close/mean_{lag}_')], axis=1)
        df[f'mean_log_returns_{lag}'] = np.mean(df.iloc[:,df.columns.str.startswith(f'log_return_{lag}_')], axis=1)
        for id in range(10):
            symbol = list_cryptocurrencies[id] 
            df[f'log_close/mean_{lag}-mean_close/mean_{lag}_{symbol}'] = np.array( df[f'log_close/mean_{lag}_{symbol}']) - np.array( df[f'mean_close/mean_{lag}']  )
            df[f'log_return_{lag}-mean_log_returns_{lag}_{symbol}'] = np.array( df[f'log_return_{lag}_{symbol}']) - np.array( df[f'mean_log_returns_{lag}'] )
    # apply TA indicators: EMA, RSI, Bollinger Bands, Stochastic Oscillator
    for id in range(10):
        symbol = list_cryptocurrencies[id]
        for lag in lags_ema:
            # df[f'ema_close_{lag}_{symbol}'] = ta.trend.ema_indicator(df[f'close_{symbol}'], window=lag, fillna=True)
            df[f'ema_close_{lag}_{symbol}'] = exponential_moving_average(df[f'close_{symbol}'], span=lag)
        for lag in lags_rsi:
            # df[f'rsi_close_{lag}_{symbol}'] = ta.momentum.rsi(df[f'close_{symbol}'], window=lag, fillna=True)
            df[f'rsi_close_{lag}_{symbol}'] = relative_strength_index(df[f'close_{symbol}'], window=lag)
        for lag in lags_bbands:
            # df[f'hbbands_close_{lag}_{symbol}'] = ta.volatility.bollinger_hband(df[f'close_{symbol}'], window=lag, fillna=True)
            # df[f'lbbands_close_{lag}_{symbol}'] = ta.volatility.bollinger_lband(df[f'close_{symbol}'], window=lag, fillna=True)
            # df[f'width_bbands_close_{lag}_{symbol}'] = ta.volatility.bollinger_wband(df[f'close_{symbol}'], window=lag, fillna=True)
            df[f'hbbands_close_{lag}_{symbol}'] = bollinger_hband(df[f'close_{symbol}'], window=lag)
            df[f'lbbands_close_{lag}_{symbol}'] = bollinger_lband(df[f'close_{symbol}'], window=lag)
            df[f'width_bbands_close_{lag}_{symbol}'] = bollinger_wband(df[f'close_{symbol}'], window=lag)
        # Alpha#12: (sign(delta(volume, 1)) * (-1 * delta(close, 1))) 
        df['alpha12_'+symbol] = np.sign(delta(df['volume_'+symbol], 1)) * (-1 * delta(df['close_'+symbol], 1))
        # Alpha#54: ((-1 * ((low - close) * (open^5))) / (0.0000001 + (low - high) * (close^5))) 
        df['alpha54_'+symbol] = ((-1 * ((df['low_'+symbol] - df['close_'+symbol]) * (df['open_'+symbol]**5))) / (0.0000001 + (df['low_'+symbol] - df['high_'+symbol]) * (df['close_'+symbol]**5)))
        # Alpha#41: (((high * low)^0.5) - close)
        # df['alpha41_'+symbol] = ((df['high_'+symbol] * df['low_'+symbol])**0.5) - df['close_'+symbol]
        # Alpha#53: (-1 * delta((((close - low) - (high - close)) / (close - low + 0.0000001)), 9))
        # df['alpha53_'+symbol] = -1 * delta((((df['close_'+symbol] - df['low_'+symbol]) - (df['high_'+symbol] - df['close_'+symbol])) / (df['close_'+symbol] - df['low_'+symbol] + 0.0000001)), 9)
        # Alpha#6: (-1 * correlation(open, volume, 10)) 
        # df['alpha6_'+symbol] = -1 * correlation(df['open_'+symbol], df['volume_'+symbol], 10)
        # Alpha#9: ((0 < ts_min(delta(close, 1), 5)) ? delta(close, 1) : ((ts_max(delta(close, 1), 5) < 0) ? delta(close, 1) : (-1 * delta(close, 1)))) 
        df['alpha9_'+symbol] = np.where(0 < ts_min(delta(df['close_'+symbol], 1), 5), delta(df['close_'+symbol], 1), np.where(ts_max(delta(df['close_'+symbol], 1), 5) < 0, delta(df['close_'+symbol], 1), -1 * delta(df['close_'+symbol], 1)))
    gc.collect()
    df.dropna(inplace=True) 
    for symbol in list_cryptocurrencies:
        df.drop(columns=[f'open_{symbol}', f'high_{symbol}', f'low_{symbol}', f'volume_{symbol}'], inplace=True)
    df.drop_duplicates(subset=['timestamp'], inplace=True)
    # df.to_csv(r'C:\Users\e0817820\Desktop\tokka\data\processed\enriched_train.csv', index=False)  
    return df   

In [13]:
def separateFeaturesForEachAsset(df):
    for id in range(10):
        symbol = list_cryptocurrencies[id]
        columns = [col for col in df.columns if symbol in col] + ['timestamp']
        df_asset = df[columns]
        df_asset.to_csv(f'C:\\Users\\e0817820\\Desktop\\tokka\\data\\processed\\train_{symbol}.csv', index=False)
        print(df_asset.shape)
        gc.collect()

In [14]:
combined_train = getFeatures(combined_train)

C:\Users\e0817820\AppData\Local\Temp\ipykernel_32020\1023688296.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'log_close/mean_{lag}-mean_close/mean_{lag}_{symbol}'] = np.array( df[f'log_close/mean_{lag}_{symbol}']) - np.array( df[f'mean_close/mean_{lag}']  )
C:\Users\e0817820\AppData\Local\Temp\ipykernel_32020\1023688296.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'log_return_{lag}-mean_log_returns_{lag}_{symbol}'] = np.array( df[f'log_return_{lag}_{symbol}']) - np.array( df[f'mean_log_returns_{lag}'] )
C

In [15]:
separateFeaturesForEachAsset(combined_train)

(1377975, 42)
(1377975, 42)
(1377975, 42)
(1377975, 42)
(1377975, 42)
(1377975, 42)
(1377975, 42)
(1377975, 42)
(1377975, 42)
(1377975, 42)


In [16]:
print(combined_train.shape)

(1377975, 419)
